In [1]:
import scipy
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential, Model

from sklearn.metrics import mean_squared_error,mean_absolute_error


In [2]:
df=pd.read_csv('San Diego_featured_data.csv')
df.head()

,All sky irradiance,Clear sky irradiance,ALLSKY ICI,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_UV_INDEX,CLRSKY_SFC_PAR_TOT
0,3.58,3.68,0.70,63.57,0.55,66.20
1,3.20,3.60,0.62,57.87,0.53,65.50
2,2.52,3.46,0.49,47.44,0.50,64.25
3,2.93,3.48,0.57,55.34,0.49,65.82
4,0.78,3.42,0.15,16.23,0.21,64.96


In [3]:
df.dropna(inplace=True)

In [4]:
y= df[['All sky irradiance']]
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(df)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten() 

In [5]:
X_train1, X_test1, Y_train, Ytest1 = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
print('Train set shape', X_train1.shape)
print('testing set shape', X_test1.shape)


Train set shape (1533, 6)
testing set shape (384, 6)


In [6]:
X_train_series=np.reshape(X_train1,(1533,6,1))
X_valid_series=np.reshape(X_test1,(384,6,1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_valid_series.shape)

Train set shape (1533, 6, 1)
Validation set shape (384, 6, 1)


In [7]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

In [8]:
model_lstm = Sequential()
model_lstm.add(GRU(128, activation='relu', return_sequences=True,input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
#model_lstm.add(LSTM(units = 72,return_sequences=True,activation="relu"))    
#model_lstm.add(LSTM(units = 72,return_sequences=True,activation="relu"))     
model_lstm.add(LSTM(units = 128,return_sequences=True,activation="relu"))
model_lstm.add(GRU(units = 128,activation="relu"))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(1,activation="linear"))
model_lstm.compile(loss='mae', optimizer=Adam(learning_rate=0.0001))
#model_lstm.summary()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
import time 
start= time.time()
lstm_history = model_lstm.fit(X_train_series, Y_train,epochs=100, batch_size=32,verbose=2)
end= time.time()
continential= end- start
print("convergence time:",continential) 

Epoch 1/100
48/48 - 2s - 38ms/step - loss: 0.7982
Epoch 2/100
48/48 - 0s - 9ms/step - loss: 0.6518
Epoch 3/100
48/48 - 1s - 11ms/step - loss: 0.1860
Epoch 4/100
48/48 - 0s - 10ms/step - loss: 0.0994
Epoch 5/100
48/48 - 0s - 10ms/step - loss: 0.0921
Epoch 6/100
48/48 - 0s - 10ms/step - loss: 0.0878
Epoch 7/100
48/48 - 0s - 10ms/step - loss: 0.0837
Epoch 8/100
48/48 - 0s - 10ms/step - loss: 0.0817
Epoch 9/100
48/48 - 0s - 10ms/step - loss: 0.0789
Epoch 10/100
48/48 - 0s - 10ms/step - loss: 0.0794
Epoch 11/100
48/48 - 0s - 10ms/step - loss: 0.0755
Epoch 12/100
48/48 - 0s - 10ms/step - loss: 0.0719
Epoch 13/100
48/48 - 0s - 10ms/step - loss: 0.0700
Epoch 14/100
48/48 - 0s - 10ms/step - loss: 0.0676
Epoch 15/100
48/48 - 0s - 10ms/step - loss: 0.0656
Epoch 16/100
48/48 - 0s - 10ms/step - loss: 0.0624
Epoch 17/100
48/48 - 0s - 10ms/step - loss: 0.0580
Epoch 18/100
48/48 - 0s - 10ms/step - loss: 0.0551
Epoch 19/100
48/48 - 0s - 10ms/step - loss: 0.0583
Epoch 20/100
48/48 - 0s - 10ms/step - los

In [10]:
import time
start= time.time()
prediction_lstm=model_lstm.predict(X_valid_series)
end= time.time()
continential= end -start
print("inference time:",continential)
prediction_lstm

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
inference time: 0.2295541763305664


array([[-1.4450542e+00],
       [-8.5443199e-01],
       [-5.3769909e-02],
       [-3.1480265e-01],
       [ 8.5911828e-01],
       [-1.2735325e+00],
       [ 4.4269717e-01],
       [-4.7999841e-01],
       [ 8.3023292e-01],
       [ 1.4630923e-01],
       [ 8.5298872e-01],
       [ 8.8974035e-01],
       [-3.9309263e-05],
       [ 9.5278722e-01],
       [ 1.1324852e+00],
       [ 8.1854254e-01],
       [ 8.7057948e-01],
       [ 1.2001740e+00],
       [-7.5876999e-01],
       [-2.6401591e-01],
       [ 4.1231939e-01],
       [ 1.0053412e+00],
       [-2.2408938e-01],
       [ 6.1627138e-01],
       [-7.6404750e-01],
       [-8.5494053e-01],
       [ 7.2284120e-01],
       [-5.2759725e-01],
       [-7.7791905e-01],
       [ 5.4588723e-01],
       [-8.6063814e-01],
       [ 2.0305160e-01],
       [ 1.3662816e+00],
       [ 1.6663855e+00],
       [ 1.5863936e+00],
       [ 4.0205330e-01],
       [ 8.4472752e-01],
       [-1.5834966e-01],
       [ 8.8225909e-02],
       [ 1.1481972e+00],


In [11]:
prediction_lstm.shape

(384, 1)

In [12]:
predictions = scaler_y.inverse_transform(prediction_lstm)
Actual = scaler_y.inverse_transform(Ytest1.reshape(-1, 1))

In [13]:
Calculated =pd.DataFrame(predictions, columns = ['Prediction'])
Actual=pd.DataFrame(Actual, columns=["Actual"])

In [14]:
import os
import pandas as pd

# Concatenate Calculated and Actual DataFrames along the columns
combined = pd.concat([Calculated, Actual], axis=1)

# Define the path to the existing folder
save_dir = "/Users/ubaidahmed/Desktop/Daily_SI_forecasting /San Diego/Models/GRU"

# Define the filename within the specified folder
filename = os.path.join(save_dir, "GRU_output.csv")

# Save the combined DataFrame to CSV
combined.to_csv(filename, index=False)

print(f"File saved to {filename}")


File saved to /Users/ubaidahmed/Desktop/Daily_SI_forecasting /San Diego/Models/GRU/GRU_output.csv


# Error Calculation

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("GRU_output.csv")
features= list(df)
features

['Prediction', 'Actual']

In [3]:
df=df*1000

In [4]:
z= df["Actual"]
x= df["Prediction"]

In [5]:
import math
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(z,x)
RMSE= math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE= mean_absolute_error(z,x)
from sklearn.metrics import mean_absolute_percentage_error
MAPE= (mean_absolute_percentage_error(z,x))*100
print("Mean Square Error is:",MSE)
print("Root Mean Square Error is:",RMSE)
print("Mean Absolute Error is:",MAE)
print("Mean Absolute Percentage Error is:",MAPE)

Mean Square Error is: 6239.166607999326
Root Mean Square Error is: 78.98839540083927
Mean Absolute Error is: 50.78135885416665
Mean Absolute Percentage Error is: 1.0827783775441706


In [6]:
max=x.max()
min=x.min()
mean=x.mean()

print("Max",max)
print("Min",min)
print('Mean',mean)

Max 8885.015
Min 1327.6485
Mean 5203.850946354167


In [7]:
NRMSE= (RMSE/(max-min))*100
print('Normalized Root Mean Square Error is:',NRMSE)

Normalized Root Mean Square Error is: 1.04518413128223


In [8]:
NRMSE= (RMSE/(mean))*100
print('Normalized Root Mean Square Error is:',NRMSE)

Normalized Root Mean Square Error is: 1.517883510022107


In [9]:
relative_errors = (z - x) / z
squared_relative_errors = relative_errors ** 2
abs_relative_errors = np.abs(relative_errors)

In [10]:
msre = np.mean(squared_relative_errors)
rmsre = np.sqrt(msre)
mare = np.mean(abs_relative_errors)
rmspe = np.sqrt(np.mean((abs_relative_errors * 100) ** 2))

In [11]:
print("Mean Square Relative Error is:",msre)
print("Root Mean Square Relative Error is:",rmsre)
print("Mean Absolute Relative Error is:",mare)
print("Root Mean Square Percentage Error is:",rmspe)

Mean Square Relative Error is: 0.00029982985349369254
Root Mean Square Relative Error is: 0.01731559567250554
Mean Absolute Relative Error is: 0.010827783775441706
Root Mean Square Percentage Error is: 1.7315595672505537
